see :
https://huggingface.co/cahya/xlm-roberta-large-indonesian-NER


In [1]:
!pip install textract transformers


In [2]:
!pip install sentencepiece

In [4]:
!apt-get install antiword


'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
def merge_ner_results(ner_results):
    merged_results = []
    current_entity = None

    for result in ner_results:
        word = result['word'].replace('▁', ' ')
        if current_entity and result['entity'].startswith('I-') and current_entity['entity_group'] == result['entity'][2:]:
            current_entity['word'] += word
            current_entity['end'] = result['end']
            current_entity['score'] = (current_entity['score'] + result['score']) / 2
        else:
            if current_entity:
                merged_results.append(current_entity)
            current_entity = {
                'entity_group': result['entity'][2:],
                'score': result['score'],
                'word': word,
                'start': result['start'],
                'end': result['end']
            }

    if current_entity:
        merged_results.append(current_entity)

    return merged_results


In [3]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel

#model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
#tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
#tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
#model = AutoModelForTokenClassification.from_pretrained("indolem/indobert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("cahya/xlm-roberta-large-indonesian-NER")
tokenizer = AutoTokenizer.from_pretrained("cahya/xlm-roberta-large-indonesian-NER")
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)
text = "Nama Saya Arief Kurniawan, saat ini tanggal 29 November 2023, di Bogor, Sedang bekerja dengan cara menghadiri rapat pada Pukul 08.00 WIB dengan membawa uang sebesar Rp 10.000.000 dan 5 buah tas."
ner_results = ner_pipeline(text)
ner_results

Some weights of the model checkpoint at cahya/xlm-roberta-large-indonesian-NER were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'B-PER',
  'score': 0.99897444,
  'index': 3,
  'word': '▁Ar',
  'start': 9,
  'end': 12},
 {'entity': 'I-PER',
  'score': 0.9829244,
  'index': 4,
  'word': 'ief',
  'start': 12,
  'end': 15},
 {'entity': 'I-PER',
  'score': 0.9992042,
  'index': 5,
  'word': '▁Kur',
  'start': 15,
  'end': 19},
 {'entity': 'I-PER',
  'score': 0.9985372,
  'index': 6,
  'word': 'ni',
  'start': 19,
  'end': 21},
 {'entity': 'I-PER',
  'score': 0.99891806,
  'index': 7,
  'word': 'awan',
  'start': 21,
  'end': 25},
 {'entity': 'B-DAT',
  'score': 0.99879634,
  'index': 12,
  'word': '▁29',
  'start': 43,
  'end': 46},
 {'entity': 'I-DAT',
  'score': 0.99955946,
  'index': 13,
  'word': '▁November',
  'start': 46,
  'end': 55},
 {'entity': 'I-DAT',
  'score': 0.99951494,
  'index': 14,
  'word': '▁2023',
  'start': 55,
  'end': 60},
 {'entity': 'B-GPE',
  'score': 0.9984962,
  'index': 17,
  'word': '▁Bogor',
  'start': 64,
  'end': 70},
 {'entity': 'B-TIM',
  'score': 0.96062624,
  'index'

In [4]:
merged_results = merge_ner_results(ner_results)
merged_results

[{'entity_group': 'PER',
  'score': 0.9978625252842903,
  'word': ' Arief Kurniawan',
  'start': 9,
  'end': 25},
 {'entity_group': 'DAT',
  'score': 0.9993464350700378,
  'word': ' 29 November 2023',
  'start': 43,
  'end': 60},
 {'entity_group': 'GPE',
  'score': 0.9984962,
  'word': ' Bogor',
  'start': 64,
  'end': 70},
 {'entity_group': 'TIM',
  'score': 0.9743636846542358,
  'word': ' 08.00 WIB',
  'start': 126,
  'end': 136},
 {'entity_group': 'MON',
  'score': 0.9993708729743958,
  'word': ' Rp 10.000.000',
  'start': 164,
  'end': 178},
 {'entity_group': 'CRD',
  'score': 0.9872613,
  'word': ' 5',
  'start': 182,
  'end': 184}]

In [5]:
# Extracting 5W1H information
who = []
when = []
where = []
what = []
how_much = []
for entity in merged_results:
    if (entity['entity_group'] == 'PER') | (entity['entity_group'] == 'ORG'):  # Assuming 'B-PER' is the label for persons
        who.append(entity['word'])
    elif (entity['entity_group'] == 'LOC') | (entity['entity_group'] == 'GPE') | (entity['entity_group'] == 'FAC') | (entity['entity_group'] == 'REG'):  # Assuming 'B-LOC' is the label for locations
        where.append(entity['word'])
    elif (entity['entity_group'] == 'TIM') | (entity['entity_group'] == 'DAT'):  # Assuming 'B-TIME' is the label for time expressions
        when.append(entity['word'])
    elif (entity['entity_group'] == 'EVT') | (entity['entity_group'] == 'PRC'):
        what.append(entity['word'])
    elif (entity['entity_group'] == 'MON') | (entity['entity_group'] == 'QTY'):
        how_much.append(entity['word'])
print("Who:", " ".join(who))
print("When", " ".join(when))
print("Where:", " ".join(where))
print("What:", " ".join(what))
print("How Much:", " ".join(how_much))

Who:  Arief Kurniawan
When  29 November 2023  08.00 WIB
Where:  Bogor
What: 
How Much:  Rp 10.000.000


In [6]:
print("Who:", " ".join(who))
print("When", " ".join(when))
print("Where:", " ".join(where))
print("What:", " ".join(what))
print("How Much:", " ".join(how_much))

Who:  Arief Kurniawan
When  29 November 2023  08.00 WIB
Where:  Bogor
What: 
How Much:  Rp 10.000.000


In [20]:
text = "A.N. Angeligue Hayyuza, Se Dan Novel Mohamad Mahri 1. Ringkasan Kasus Angeligue Hayyuza (Ah) Dilaporkan Oleh Pedagang Valuta Asing, Pt Dinamis Citra Swakarsa Terkait Transaksi Penjualan Bank Notes Dalam Jumlah Besar. Berdasarkan Informasi Pt Dinamis Citra Swakarsa, Ah Berprofesi Sebagai Pegawai Swasta Namun Tidak Diketahui Nama Dan Domisili Perusahaan Tempat Ybs Bekerja. Sesuai Data Nasabah Dari Bank Bca Terinformasi Bahwa Ah Berprofesi Sebagai Pedagang (Istri Pemilik Toko Himalaya-Toko Bangunan Berdasarkan Data Bank Mandiri, Pekerjaan Ah Adalah Ibu Rumah Tanggaterinformasi Juga Media Online, Sejak Tahun 2012, Ah Adalah Pegawai Pada Perusahaan Money Remittance (Nusantara Money Remittance) Yang Berdomisili Operasi Di Sydney Australiaselama Periode Mei 2012 S/D Juli 2013 (15 Bulan Teridentifikasi Ah Melakukan 40 Kali Transaksi Penjualan Bank Notes Dengan Total Nominal Aud 571,447 (Eg Rp5.308.610.800)Atau Rata-Rata Ah Melakukan 3 Kali Transaksi Per Bulanah Memiliki 3 (Tiga) Rekening Di Bank Mandiri Dan Bank Bca. Berdasarkan Mutasi Rekening Bca Nomor 2810297291 A.N Ybs, Selama Periode Januari 2012 S/D April 2013, Ah Menerima Aliran Dana Sebanyak 443 Kali Sejumlah Total Rp29.663.595.676, Termasuk Setoran Tunai Senilai Total Rp718.710.000. Adapun Aliran Dana Keluar Berupa 1.872 Kali Transaksi Pengiriman Dana Total Rp17.434.835.607. (Rata-Rata Sebesar Rp9.313.480 Per Transaksikepada Banyak Pihak Dengan Underlying Nusantara Diduga Nusantara” Adalah Nusantara Money Transfer, Perusahaan Tempat Ybs Bekerjaberdasarkan Mutasi Rekening Bank Mandiri Nomor 9000003185833, Pada Periode Januari 2012 S/D Maret 2013, Ah Aktif Menerima 102 Kali Transaksi Aliran Dana Berupa Pemindahbukuan Dari Seseorang Bernama Novel Mohamad Mahri (Nmm) Dengan Total Nominal Rp3.213.127.075. Adapun Aliran Dana Keluar Berupa 855 Kali Transaksi Pengiriman Dana Total Rp6.922.841.034,Pnii Ii Gi N Ii Magi I A Ama Ai Gnii Inng Pennan/ E Pn Jii Mgii Jcugiiixiiyg. Peni Na Ii Nga Si Vv Ner/ Pe Aag Iiiciiycuulixgiitransaksi Pada Kanggaru Bisa Menggunakan 4 (Empat) Rekening Bank Atas Nama Nmm Masing-Masing Di Bank Mandiri, Bank Bca, Bank Bni, Bank Briselama Periode 01 Januari 2012 S/D 30 Maret 2013, Melalui Rekening Bank Mandiri Nomor 1400004357068, Nmm Menerima Aliran Dana Masuk Yang Berasal Dari Setoran Tunaioverbooking, Rtgs Dll Dari Beberapa Pihak Sebanyak 2.423 Kali Transaksi Dengan Total Nominal Rp152.345.554.931. Dari Jumlah Tersebut, Aliran Dana Yang Berasal Dari Ybs Sendiri (Nmm) Mencapai 202 Kali Transaksi Berupa Setoran Tunai Dan Rtgs, Total Nominal Rp50.910.900.000. Teridentifikasi Untuk Aliran Dana Keluar, Nmm Mengirimkan Dana Ke Banyak Pihak (13.956 Kali Transaksi, Total Nominal Rp151.375.094.282 Berdasarkan Pola Mutasi (Debit Vs Kredit) Transaksi Yang Bersifat From-Many-To-Many” Diindikasikan Terkait Dengan Kegiatan Remitansi Pada Kupu Kanggaruinformasi Dari Website Www.Kanggaru.Net, Kanggaru Merupakan Perusahaan Terdaftar Di Remitansi Kupu Kanggaru Dan Jaringannya Di Indonesia. Seharusnya, Kupu Kanggaru Menggunakan Rekening Perusahaan Untuk Transaksi Remitansi Tersebutatas Dasar Pertimbangan Tersebut Maka Hasil Analisis Atas Nama Angeligue Hayyuza Dan Novel Mohamad Mahri Diteruskan Kepada Direktorat Jenderal Pajak Untuk Ditindaklanjuti Sesuai Kewenangan Yang Berlaku Serta Menjadi Informasi Kepada Bank Indonesia Terkait Fungsi Pengawasan Dan Pembinaan Terhadap Industri Kupu 2. Profil Terlapor Dan Terkait Serta Pola Transaksi A. Profil Terlapor Nama : | Angeligue Hayyuza (Ahpep : | Tidak Jenis Kelamin : | Perempuan Tempat, Tgl Lahir : | 29 November 1984 Alamat : | Jl Saturnus Utara Ix No 1 Rt 02 Rw 11 Munjahlega Rancasari Bandung Jawa Barat 40286 Pekerjaan : | Ibu Rumah Tangga (Data Nasabah Bank Mandiripegawai Swasta (Karyawan Nusantara Money Transfer Perusahaan Money Remittance) — Informasi Dari Media Online Pedagang (Istri Pemilik Toko Himalaya-Toko Bangunan) Data Nasabah Bca Identitas Diri : | Ktp No. 1050216911843001 Alamat Tempat Kerja Nusantara Money Transfer, 52/3-7Fetherstone St Bca Nomor 0080957700 A.N Angeligue Hayyuza Bank Mandiri 9000003185833 A.N Angeligue Hayyuza Nama : | Novel B Mohammad Mahri (Nmmpep : | Tidak Jenis Kelamin : | Laki-Laki Kepemilikan Dana : | Pemilik Dana Kewarganegaraan : | Indonesia Tempat, Tgl Lahir : | Surabaya, O6 November 1962 Alamat : | Ampel Wirai 2/4 Rt 002 Rw 003, Ampel Lonceng 27Ampel, Semampir Surabaya - Jawa Timur 60151 No Telp 081553793218 Email Nmahri@Optusnet.Com.Au Alamat Tempat Kerja Kanggaru Money Transfer, Surabaya Jawa Timur Norek Yang Digunakan Bank Mandiri 1400004357068 A.N Nmm Bca 0360198774 A.N Nmm Bni 0120862977 A.N Nmm Bri 105 101000 081 503 A.N Nmm Nama Nusantara Money Transfer (Nmtbidan Nin Fe Docwne ( Was Created Wiih Spile.Doc For Py Registrasi Fair Trading Registration : Bn 9855 Pendiri Muhammad Basuki Alamat Surat 52/3-7Fetherstone St, Bankstone, Nsw Telp 0401-637278 Email Info@Mynusantara.Com Remittance Rekanan Di Pt Warindo Indonesia Nama Pt Warindo (Wabidang Usaha Pengiriman Uang (Mmoney Remittancealamat Graha Mas Fatmawati A-30, Ji Rs. Fatmawati No.71 Jakarta 12150 Honemail : | Pt.Warindo@Yahoo.Com Website : | Http:Www.Warindo.Co.Id Nama : | Kanggaru Money Transfer Bidang Usaha : | Pengiriman Uang (Mmoney Remittanceregistrasi : | Abn(770792 937 159) Dan Can (117 567 110Pendiri Ie Alamat Di Australia : | 11Marsham St. Noble Park Victoria 3174 Australia Telp : | 61402181266 /4 Surabaya 60151 Telp : | 031-3550844, 031-3545708, 62 815537932183 Email : | Thahri@Kanggaru.Net B. Pola Transaksi 1. Berdasarkan Data Pt Dinamis Citra Swakarsa, Selama Periode Mei 2012 S/D Juli 2013 (15 Bulan Ah Melakukan 40 Kali Transaksi Penjualan Bank Notes Dengan Rincian Sbb: Transaksi Jumlah Rupiah N Tgl T Ksi Rekeni I K ) Gi Transaksi Valas (Rp) Ekening Yang Digunakan Bca 2810297291 1 1 31/07/2013 Aud 19,195 178.513.500,Mandiri 9000003185833 2 24/07/2013 Aud 13,145 123.225.000 | Bca 2810297291 3 18/07/2013 Aud 18,360 169.802.250 | Bca 2810297291 4 16/07/2013 Aud 14,870 136.051.350 | Bca 2810297291 5 /109/07/2013 Aud 32,110 290.577.400 | Bca 2810297291 E— Nij Na Nj G N Da Gi P Maa M — Ie M A M My 7 I 8 Om 7 A / Om T S Gu O 7 | 19/06/2013 Aud 11,125 103.462.500 | Mandiri 9000003185833 Bca 2810297291 8 | 31/05/2013 Aud 12,465 117.756.400,Mandiri 9000003185833 9 / 06/05/2013 Aud 24,045 239.237.800 | Bca 2810297291 10 | 01/05/2013 Aud 15,515 155.130.000 | Bca 2810297291 11 | 08/03/2013 Aud 11,791 116.436.125 | Bca 2810297291 12 | 28/02/2013 Aud 10,112 99.856.000 | Bca 2810297291 13 | 15/01/2013 Aud 14,023 144.436.900 | Bca 2810297291 14 | 14/01/2013 Aud 15,705 162.154.125 | Bca 2810297291 Bca 2810297291 15 | 09/01/2013 Aud 9,543 97.577.175,Mandiri 9000003185833 Bca 2810297291 16 | 08/01/2013 Aud 14,331 146.176.200,Mandiri 9000003185833 : The Document Was Created With &Elre'Doc For Python21/12/2012 2810297291 17 Aud 10,471 105.757.100, Mandiri 9000003185833 Bca 2810297291 18 | 20/12/2012 Aud 4,933 49.823.300,Mandiri 9000003185833 Bca 2810297291 19 | 19/12/2012 Aud 6,483 65.478.300,Mandiri 9000003185833 Bca 2810297291 20 | 17/12/2012 Aud 5,984 61.347.400,Mandiri 9000003185833 Bca 2810297291 21 | 14/12/2012 Aud 11,975 120.947.500 Mandiri 9000003185833 Bca 2810297291 22 | 13/12/2012 Aud 28,310 7171.689.200,Mandiri 9000003185833 Bca 2810297291 23 | 12/12/2012 Aud 19,654 198.505.400,Mandiri 9000003185833 Bca 2810297291 24 | 11/12/2012 Aud 19,602 197.000.100 Mandiri 9000003185833 25 | 06/12/2012 Aud 8,834 88.340.000 Bca 2810297291 Mandiri 9000003185833 7 I 8 Om 7 A / Om T S Gu O 26 | 05/12/2012 Aud 13,765 137.650.000,Mandiri 9000003185833 27 | 04/12/2012 Aud 5,497 54.695.150,00 | Mandiri 9000003185833 28 | 03/12/2012 Aud 5,098 50.725.100 | Bca 2810297291 Bca 2810297291 29 | 21/11/2012 Aud 10,592 105.390.400,Mandiri 9000003185833 Bca 2810297291 30 | 20/11/2012 Aud 15,565 154.871.750,Mandiri 9000003185833 Bca 2810297291 31 | 19/11/2012 Aud 15,879 157.202.100 | N Ar 9000003185833 Bca 2810297291 32 | 12/11/2012 Aud 12,012 119.519.400 | N Ar 9000003185833 33 | 13/11/2012 Aud 15,970 158.901.500 | Bca 2810297291 34 | 07/08/2012 Aud 21,571 213.552.900 | Bca 2810297291 . . . %A 2810297291 Bca 2810297291 36 | 20/06/2012 Aud 21,612 205.314.000 | Na 9000003185833 37 | 19/06/2012 Aud 12.216 115.441.200 | Mandiri 9000003185833 38 | 18/06/2012 Aud 12,852 121.451.400 | Bca 2810297291 39 | 28/05/2012 Aud 12,834 117.431.100 | Bca 2810297291 40 | 23/05/2012 Aud 9,945 91.742.625 | Bca 2810297291Total 571,447 5.308.610.800,2. Selama Periode Mei 2012 S/D Juli 2013 (15 Bulan) Dengan Total Nominal Aud 571,447 (Eg Rp5.308.610.800 Rata-Rata Ah Melakukan 3 Kali Transaksi Per Bulan3. Ah Diketahui Memiliki Tiga Rekening Di Bca, Namun Hanya Dua Rekening Yang Masih Aktif. Ah Memiliki Fasilitas Kartu Kredit Yang Dibuka Sejak Tanggal 3 Oktober 2012 Dengan Limit Kredit Sebesar Rp5.000.000, Namun Ybs Tidak Memiliki Fasilitas Pinjaman, Sdb Dan Ybs Juga Bukan Merupakan Nasabah Kustodian Di Bca. Rincian Kepemilikan Rekening Yang Dimiliki Ah Sbb: 2800539881 Bca Kco Kiaracondong | 16/06/2005 19/05/2010 2810297291 Bca Kcp Rajawali 12/12/2011 39.670.698,47 0080957700 Bca Kcp Rajawali 07/05/2013 1.472.091,72 Berdasarkan Mutasi Rekening Nomor 2810297291, Selama Periode Januari 2012 S/D April 2013, Ah Menerima Dana Masuk Dari Pt Dinamis Citra Swakarsa (Perusahaan Money Changer) Sebanyak 65 Kali Transaksi, Total Rp6.535.337.505. Ah Juga Menerima Dana Dari Novel M Mahri Sebanyak 30 Kali Transaksi Total Sebesar Rp1.036.571.715. Diduga Pengiriman Ini Terkait Dengan Bisnis Yang Dimiliki Nmm Yaitu Bisnis Pengiriman Uang (Mmoney Remittance)4. Sementara Aliran Dana Keluar Dari Rekening Ah Berupa Transaksi Pengiriman Dana Ke Banyak Pihak Dengan Underlying Nusantara Diduga Nusantara” Merupakan Nusantara Money Transfer, Perusahaan Tempat Ybs Bekerja. Selama Periode Januari 2012 S/D April 2013 (16 Bulan Ah Mentransfer Dana Sebangk 1.872 Kali Dengan Total Ire.Doc For Xthondana Sebesar Rp17.434.835.607. (Rata-Rata Sebesar Rp9.313.480 Per Transaksi)Aliran Dana Keluar Dari Rekening Ah Lainnya Berupa Transfer, Pembelian Barang, Tarikan Tunai Sebanyak 149 Kali Dengan Total Rp2.776.348.200Transaksi Aliran Dana Keluar Dari Rekening Bca Nomor 2810297291 Ah Periode Januari 2012 S/D April 2013 Dengan Dengan Nominal Lebih Dari Rp50.000.000 Sbb: D Tgl / Jenis Transaksi Jumlah Nama Pengirim/Penerima Keterangan Berita K Pemindahan Novel Mohamad 1-Feb-12 | K | Langsung Dari 268,403,600.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Langsung 1-Feb-12 | D | Tabungan Ke 94,500,000.00 | Pwe Tjhai Trsf Ke 0290336457 | Pwe Tjhai Tahapan Pemindahan Wierman Liduan Wierman 1-Feb-12 | D | Langsung 94,500,000.00 | (Pemilik Toko Naga Trsf Ke 6430057799 | Liduan Tabungan Ke Giro Komodo - Alat Rumah Tanggapemindahan Novel Mohamad 2-Feb-12 | K | Langsung Dari 323,273,900.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 3-Feb-12 | K | Langsung Dari 326,790,450.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Langsung 3-Feb-12 | D | Tabungan Ke 94,500,000.00 | Gui Lie Tju Trsf Ke 0931070508 | Guilie Tju Tahapan Pemindahan Langsung 3-Feb-12 | D | Tabungan Ke 94,500,000.00 | Pwe Tjhai Trsf Ke 0290336457 | Pwe Tjhai Tahapan Pemindahan Novel Mohamad 6-Feb-12 | K | Langsung Dari 180,774,975.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Wierman Liduan Wierman 6-Feb-12 | D | Langsung 142,875,000.00 | (Pemilik Toko Naga Trsf Ke 6430057799 | Liduan Tabungan Ke Giro Komodo - Alat Rumah Tanggatarikan Tunai 6-Feb-12 | D | Dgnbuku Kecil 53,676,000.00 (Thp Goldpemindahan Novel Mohamad 7-Feb-12 | K | Langsung Dari 122,409,000.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 . . Hapan I 8-Feb-12 | K | Langsung Dari 133,868,950.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 10-Feb-12 | K | Langsung Dari 163,382,350.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Wierman Liduan Wierman 10-Feb-12 | D | Langsung 95,500,000.00 | (Pemilik Toko Naga Trsf Ke 6430057799 | Liduan Tabungan Ke Giro Komodo - Alat Rumah Tanggadindin B Zakaria (Karyawan Ptsrikania 10-Feb-12 | D | Iransfer Via Ib 57,300,000.00 | Cimandiri - Nusantara Ke Tahapan Engineering Automotive Partspemindahan Novel Mohamad 13-Feb-12 | K | Langsung Dari 58,316,150.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dsetoran 14-Feb-12 | K | Pemindahandgn | 141,288,700.00 Berita Pemindahan Novel Mohamad 16-Feb-12 | K | Langsung Dari 106,258,400.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dtransfer Via Ib 16-Feb-12 | D | Ke Tahapan 50,302,500.00 | Budi Horno Teja Ke Nusantara Pemindahan Novel Mohamad 20-Feb-12 | K | Langsung Dari 152,453,150.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&D22-Feb-12 | K | Langsung Dari 99,199,000.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 24-Feb-12 | K | Langsung Dari 88,406,400.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 27-Feb-12 | K | Langsung Dari 73,152,000.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 28-Feb-12 | K | Langsung Dari 98,402,150.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 29-Feb-12 | K | Langsung Dari 80,212,000.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 2-Mar-12 | K | Langsung Dari 211,687,100.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Langsung Ricy 2-Ma-12 | D | Tabungan Ke 72,750,000.00 | Ricy Candra Trsf Ke 3360020648 | Candra Tahapan Pemindahan Novel Mohamad 5-Mar-12 | K | Langsung Dari 153,987,600.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 . . Hapan 5-Ma-12 | D %Zggflgxg Ke 130,450,000.00 Trsf Ke 4740620121 Mtjjsaunrt Tahapan Karjanti Mansur (Ibu Rumah Tanggakoreksi Tarikan 5-Mar-12 | K | Pemindahan Tnp 67,725,000.00 Sal Nominal 627 25000 Buku Transfer Via Ib 5-Ma-12 | D | Ee Tahapan 57,900,000.00 | Gui Lie Tju Nusantara Aus Pemindahan Novel Mohamad 7-Mar-12 | K | Langsung Dari 60,703,900.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 9-Mar-12 | K | Langsung Dari 75,861,200.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 13-Mar-12 | K | Langsung Dari 56,756,450.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 14-Mar-12 | K | Langsung Dari 89,925,100.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dtransfer Via Ib Karjanti Mansur (Ibu 14-Mar-12 D 61,120,000.00 Nusantara Aus Ke Tahapan Rumah Tanggapemindahan Novel Mohamad 16-Mar-12 | K | Langsung Dari 53,277,750.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&Dpemindahan Novel Mohamad 19-Mar-12 | K | Langsung Dari 123,086,000.00 | Mahri (Pemilik Toko Trsf Dr 4681175525 Tahapan Barokah - P&D10"
ner_results = ner_pipeline(text)
ner_results

[{'entity': 'B-PER',
  'score': 0.99569535,
  'index': 1,
  'word': '▁A',
  'start': 0,
  'end': 1},
 {'entity': 'I-PER',
  'score': 0.58344275,
  'index': 2,
  'word': '.',
  'start': 1,
  'end': 2},
 {'entity': 'I-PER',
  'score': 0.65746254,
  'index': 3,
  'word': 'N',
  'start': 2,
  'end': 3},
 {'entity': 'I-PER',
  'score': 0.8147393,
  'index': 4,
  'word': '.',
  'start': 3,
  'end': 4},
 {'entity': 'I-PER',
  'score': 0.9260153,
  'index': 5,
  'word': '▁Angeli',
  'start': 4,
  'end': 11},
 {'entity': 'I-PER',
  'score': 0.9918264,
  'index': 6,
  'word': 'gue',
  'start': 11,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.9969343,
  'index': 7,
  'word': '▁Hay',
  'start': 14,
  'end': 18},
 {'entity': 'I-PER',
  'score': 0.9966725,
  'index': 8,
  'word': 'y',
  'start': 18,
  'end': 19},
 {'entity': 'I-PER',
  'score': 0.99659413,
  'index': 9,
  'word': 'uza',
  'start': 19,
  'end': 22},
 {'entity': 'B-PER',
  'score': 0.6766662,
  'index': 11,
  'word': '▁Se',
  'star

In [21]:
merged_results = merge_ner_results(ner_results)
merged_results

[{'entity_group': 'PER',
  'score': 0.9870443451218307,
  'word': ' A.N. Angeligue Hayyuza',
  'start': 0,
  'end': 22},
 {'entity_group': 'PER',
  'score': 0.6766662,
  'word': ' Se',
  'start': 23,
  'end': 26},
 {'entity_group': 'PER',
  'score': 0.9658958464860916,
  'word': ' Novel Mohamad Mahri',
  'start': 30,
  'end': 50},
 {'entity_group': 'PER',
  'score': 0.9961642473936081,
  'word': ' Angeligue Hayyuza',
  'start': 69,
  'end': 87},
 {'entity_group': 'PER',
  'score': 0.8584772,
  'word': 'Ah',
  'start': 89,
  'end': 91},
 {'entity_group': 'ORG',
  'score': 0.9984800284728408,
  'word': ' Pt Dinamis Citra Swakarsa',
  'start': 131,
  'end': 157},
 {'entity_group': 'PRD',
  'score': 0.8765986561775208,
  'word': ' Bank Notes',
  'start': 185,
  'end': 196},
 {'entity_group': 'ORG',
  'score': 0.9984336476773024,
  'word': ' Pt Dinamis Citra Swakarsa',
  'start': 238,
  'end': 264},
 {'entity_group': 'PER',
  'score': 0.9970831,
  'word': ' Ah',
  'start': 265,
  'end': 268

In [22]:
# Extracting 5W1H information
who = []
when = []
where = []
what = []
how_much = []
for entity in merged_results:
    if (entity['entity_group'] == 'PER') | (entity['entity_group'] == 'ORG'):  # Assuming 'B-PER' is the label for persons
        who.append(entity['word'])
    elif (entity['entity_group'] == 'LOC') | (entity['entity_group'] == 'GPE') | (entity['entity_group'] == 'FAC') | (entity['entity_group'] == 'REG'):  # Assuming 'B-LOC' is the label for locations
        where.append(entity['word'])
    elif (entity['entity_group'] == 'TIM') | (entity['entity_group'] == 'DAT'):  # Assuming 'B-TIME' is the label for time expressions
        when.append(entity['word'])
    elif (entity['entity_group'] == 'EVT') | (entity['entity_group'] == 'PRC'):
        what.append(entity['word'])
    elif (entity['entity_group'] == 'MON') | (entity['entity_group'] == 'QTY'):
        how_much.append(entity['word'])
print("Who:", " ".join(who))
print("When", " ".join(when))
print("Where:", " ".join(where))
print("What:", " ".join(what))
print("How Much:", " ".join(how_much))

Who:  A.N. Angeligue Hayyuza  Se  Novel Mohamad Mahri  Angeligue Hayyuza Ah  Pt Dinamis Citra Swakarsa  Pt Dinamis Citra Swakarsa  Ah  Bank Bca  Ah To  Bank Mandiri  Ah  Media  Ah  Money Remittance Nusantara Money Remittance  Ah  Ah  Bank Mandiri  Bank Bca  Ah  Nusantara  Nusantara Money Transfer  Ah  Novel Mohamad Mahri Nmm P ni
When  2012,  Mei 2012 Juli 2013  Januari 2012  April 2013,  Januari 2012  Maret 2013,
Where:  Sydney Australia  Nusantara
What: 
How Much:  (15 Bulan  A ud 571,447  Rp5.308.610.800  Rp29.663.595.676  Rp718.710.000  Rp17.434.835.607  Rp9.313.480  Rp3.213.127.075  Rp6.922.841.034
